In [1]:
%load_ext autoreload
%autoreload 2


In [5]:
import inept
import torch


In [ ]:
# Algorithm v1
# Find some way to encode variable-length lists (maybe encode recurrently, or by attention)
# Each node has vision range which is encoded as above
# Decides on movement, reruns
# Maybe use PPO and masked self-attention like in https://openai.com/research/emergent-tool-use

# Algorithm v2
# Self-attention encoding of other node positions to NN (no LSTM) for clipped x, y adjustment
# PPO for training with reward based on node similarity (another NN) vs proximity
# Node similarity objective is user-provided
# PROBLEM: Too much calculation for node similarity (N^2 complexity)


In [ ]:
# Params
num_nodes = 100
num_dims = 2

# Data
M1 = torch.rand((num_nodes, 2000))
M2 = torch.rand((num_nodes, 2000))


In [29]:
# Make individual node models
nodes = [inept.models.Node(2*num_dims, num_dims) for _ in range(num_nodes)]

# Randomize starting states
# x, y, vx, vy
node_features = torch.rand((num_nodes, 2*num_dims))

# Run model over all nodes
for i, node in enumerate(nodes):
    idx = torch.zeros((node_features.shape[0]), dtype=torch.bool)
    idx[i] = True
    node(node_features[idx], node_features[~idx])

# TODO: PPO
